# Prompt
สร้าง dynamic prompt โดยแบ่งแยกโดยคะแนนที่น้องทำได้

In [131]:
def generate_prompt_from_row(row):
    # Skip processing if INCORRECT_SUBTOPIC_LIST is empty or null for scores of 0
    if row['SCORE_PCT'] == 0 and (pd.isna(row['INCORRECT_SUBTOPIC_LIST']) or row['INCORRECT_SUBTOPIC_LIST'].strip() == ""):
        return None

    if row['SCORE_PCT'] == 0:
        # Tailored prompt for students who scored 0 but have areas to improve, in Thai
        return f"""
        ### CONTEXT:
        ให้คำแนะนำสำหรับนักเรียนที่เริ่มต้นด้วยความท้าทายอย่างมาก เนื่องจากพวกเขาได้คะแนน 0 การให้การสนับสนุนและแนวทางในหัวข้อพื้นฐานเพื่อการปรับปรุงเป็นสิ่งสำคัญ
        
        ### TASK:
        - พิจารณาความท้าทายที่พบและให้คำแนะนำที่สนับสนุนและสามารถดำเนินการได้จริงเกี่ยวกับหัวข้อใน INCORRECT_SUBTOPIC_LIST นี้ [{row['INCORRECT_SUBTOPIC_LIST']}] เพื่อช่วยให้พวกเขาเริ่มต้นการเดินทางสู่ความเข้าใจและคะแนนที่สูงขึ้น
        - Include all INCORRECT_SUBTOPIC_LIST in your advice.
        
        ### OUTPUT:
        - คำแนะนำสั้นๆที่ชัดเจนและกระชับ ไม่เกิน 30 คำ
        - ใช้ภาษาไทย
        """
    elif row['SCORE_PCT'] == 1:
        # Tailored prompt for students who scored 1, in Thai
        return """
        ### CONTEXT:
        ขอแสดงความยินดีด้วยที่คุณทำได้คะแนนเต็ม! แม้ว่าคุณจะบรรลุความสำเร็จในระดับสูงสุด แต่อย่าลืมว่าการเรียนรู้เป็นเส้นทางที่ไม่มีที่สิ้นสุดและยังมีพื้นที่สำหรับการเติบโตต่อไป
        
        ### TASK:
        - แม้คุณจะได้คะแนนเต็มแล้ว เรายังต้องการให้คำแนะนำเพิ่มเติมเพื่อช่วยเสริมความรู้และทักษะของคุณ
        - ให้คำแนะนำที่สนับสนุนและกระตุ้นใจสำหรับการปรับปรุงและการเติบโตต่อไป
        
        ### OUTPUT:
        - คำแนะนำสั้นๆที่ชัดเจนและกระชับ ไม่เกิน 30 คำ
        - ใช้ภาษาไทย
        """
    elif 0.61 <= row['SCORE_PCT'] < 1:
        # Tailored prompt for scores between 0.61 to 0.99, in Thai
        return f"""
        ### CONTEXT:
        ให้คำแนะนำสำหรับนักเรียนที่ทำได้ดีแล้ว แต่ยังมีเส้นทางให้พัฒนาเพื่อบรรลุความสมบูรณ์แบบ เราต้องการช่วยให้พวกเขาเห็นคุณค่าของการปรับปรุงอย่างต่อเนื่อง
        
        ### TASK:
        - พิจารณาหัวข้อใน INCORRECT_SUBTOPIC_LIST นี้ [{row['INCORRECT_SUBTOPIC_LIST']}] ที่พวกเขายังสามารถเรียนรู้เพื่อปรับปรุงได้ เพื่อช่วยให้พวกเขาก้าวหน้าและเพิ่มคะแนนให้สูงขึ้น
        - ให้คำแนะนำที่สนับสนุนและกระตุ้นใจสำหรับการปรับปรุง
        - Include all INCORRECT_SUBTOPIC_LIST in your advice.
        
        ### OUTPUT:
        - คำแนะนำสั้นๆที่ชัดเจนและกระชับ ไม่เกิน 30 คำ
        - ใช้ภาษาไทย
        """
    else:
        # Tailored prompt for scores below 0.61, in Thai
        return f"""
        ### CONTEXT:
        ให้คำแนะนำสำหรับนักเรียนที่ต้องการการสนับสนุนเพิ่มเติม เนื่องจากพวกเขาได้คะแนนต่ำกว่า 0.61 เราต้องการให้พวกเขารู้สึกว่ายังมีโอกาสในการปรับปรุงและเราพร้อมที่จะช่วยเหลือ
        
        ### TASK:
        - พิจารณาหัวข้อใน INCORRECT_SUBTOPIC_LIST นี้ [{row['INCORRECT_SUBTOPIC_LIST']}] ที่ต้องการความช่วยเหลือเพิ่มเติมในการเรียน เพื่อเพิ่มคะแนนและปรับปรุงความเข้าใจ
        - สร้างคำแนะนำที่สร้างสรรค์และมีประโยชน์
        - Include all INCORRECT_SUBTOPIC_LIST in your advice.
        
        ### OUTPUT:
        - คำแนะนำสั้นๆที่ชัดเจนและกระชับ ไม่เกิน 30 คำ
        - ใช้ภาษาไทย
        """

# load ข้อมูล

In [ ]:
# Ensure to use the correct path to your CSV file
df = pd.read_csv(r'C:\Users\nuttapong.but\Downloads\MedSchool_SuggestinList.csv')

# OpenAI

#### สร้างฟังชั่นเพื่อใช้โมเดล

In [132]:
import openai
import pandas as pd

# Set your OpenAI API key here
openai.api_key = 'replace with your api key'

def query_openai_model(prompt, model="gpt-3.5-turbo-0125", temperature=0):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": """
                        - เพศหญิง
                        - คุณครู
                        - ใส่ใจ
                        - แทนตัวเองว่า 'พี่'
                        - แทนนักเรียนว่า 'น้อง'
                    """},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message['content']
    except Exception as e:
        return str(e)


#### loop กับขั้อมูล และ ส่งออกเป็น csv file

In [133]:
# Process rows as per your requirements
for index, row in df.iterrows():
    prompt = generate_prompt_from_row(row)
    if prompt is None:
        continue  # Skip processing this row
    suggestion = query_openai_model(prompt, temperature=0)
    df.at[index, 'Suggestions'] = suggestion

# Save the updated DataFrame
df.to_csv(r'C:\Users\nuttapong.but\Downloads\MedSchool_SuggestionList_OpenAI.csv', index=False, encoding='utf-8-sig')

# Anthropic

In [108]:
pip install anthropic

   ---------------------------------------- 0.0/851.6 kB ? eta -:--:--
   -- ------------------------------------- 61.4/851.6 kB 1.1 MB/s eta 0:00:01
   --------- ------------------------------ 204.8/851.6 kB 2.5 MB/s eta 0:00:01
   --------------- ------------------------ 337.9/851.6 kB 2.6 MB/s eta 0:00:01
   ---------------------- ----------------- 471.0/851.6 kB 2.7 MB/s eta 0:00:01
   --------------------------------- ------ 706.6/851.6 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 851.6/851.6 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/75.6 kB ? eta -:--:--
   ---------------------------------------- 75.6/75.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/77.9 kB ? eta -:--:--
   ---------------------------------------- 77.9/77.9 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------- ----------------------------- 0.6/2.2 MB 17.8 MB/s eta 0:00:01
   ------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#### สร้างฟังชั่นเพื่อใช้โมเดล

In [134]:
client = anthropic.Anthropic(
    api_key="replace with your api key",
)

def query_anthropic_model(prompt, model="claude-3-haiku-20240307", temperature=0):
    try:
        # Construct the system preferences string according to your requirements
        system_preferences = """
            - เพศหญิง
            - คุณครู
            - ใส่ใจ
            - แทนตัวเองว่า 'พี่'
            - แทนนักเรียนว่า 'น้อง'
        """
        
        # Create a message with the Anthropics client
        message = client.messages.create(
            model=model,  # Specify the model [claude-3-opus-20240229, claude-3-sonnet-20240229, claude-3-haiku-20240307]
            max_tokens=4096,
            temperature=temperature,
            system=system_preferences,  # Apply the system preferences
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        # Return the 'text' content of the first ContentBlock in the response
        return message.content[0].text if message.content else "No response text."
    except Exception as e:
        # Return the exception message in case of an error
        return str(e)

# Example usage of the function
#response = query_anthropic_model("Hi Claude.", model="claude-3-haiku-20240307", temperature=0)
#print(response)

#### loop กับขั้อมูล และ ส่งออกเป็น csv file

In [135]:
# Process rows as per your requirements
for index, row in df.iterrows():
    prompt = generate_prompt_from_row(row)
    if prompt is None:
        continue  # Skip processing this row
    suggestion = query_anthropic_model(prompt, temperature=0)
    df.at[index, 'Suggestions'] = suggestion

# Save the updated DataFrame
df.to_csv(r'C:\Users\nuttapong.but\Downloads\MedSchool_SuggestionList_Anthropic.csv', index=False, encoding='utf-8-sig')

# GEMINI

In [62]:
import requests

# Set your Gemini API key here
API_KEY = 'replace with your api key'
# Specify the Gemini endpoint URL, including the API key in the query parameter
ENDPOINT_URL = 'https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent'

def get_access_token(client_id, client_secret):
    # This URL might change based on Google's documentation
    token_url = "https://oauth2.googleapis.com/token"
    payload = {
        "client_id": client_id,
        "client_secret": client_secret,
        "scope": "scope_of_the_gemini_api",
        "grant_type": "client_credentials",  # This might vary
    }
    response = requests.post(token_url, data=payload)
    return response.json().get("access_token")
    

# Define a function to query the Gemini model
def query_gemini_model(prompt, temperature=0):
    # Headers for the API request
    headers = {
        'Authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json',
    }
    # Define the data to be sent in the API request
    data = {
        'prompt': {
            'text': prompt  # Adjust based on the Gemini API's expected prompt format
        },
        'temperature': temperature,
        # Add additional parameters here as required by the Gemini API
    }
    try:
        # Make the API request, appending the API key to the URL
        response = requests.post(f'{ENDPOINT_URL}?key={API_KEY}', headers=headers, json=data)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response JSON and return the desired information
            # Adjust this based on the actual structure of Gemini's response
            return response.json()['responses'][0]['text']  # Example path to the generated text
        else:
            return f'Error: Received response code {response.status_code}, Message: {response.text}'
    except Exception as e:
        return f'Exception occurred: {str(e)}'

# Example usage
prompt = "What is the meaning of life?"
response = query_gemini_model(prompt, temperature=0.5)
print(response)


Error: Received response code 401, Message: {
  "error": {
    "code": 401,
    "message": "Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.",
    "status": "UNAUTHENTICATED"
  }
}

